<a href="https://colab.research.google.com/github/zhibor/notebooks/blob/main/mistral_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mistral v0.3: Multi-Function Calling & Dependencies! 🚀

Mistral just dropped version 0.3, and it natively supports function calling! 🛠️

I tested multi-functions & dependencies. It didn't work initially, but I found a solution that works like a charm. Check out my video and try it on Colab. Enjoy!

Watch video here: https://youtu.be/MZlRaclJ4YI

Follow me on X: https://x.com/unclecode

In [ ]:
%%capture
!pip install mistral_inference

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/root/mistral_models/7B-Instruct-v0.3'

In [ ]:
from mistral_common.protocol.instruct.tool_calls import Function, Tool
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, SystemMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
model = Transformer.from_folder(mistral_models_path)

completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_weather",
                description="Get the current weather",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "format": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["location", "format"],
                },
            )
        ),
        Tool(
            function=Function(
                name="get_current_time",
                description="Get the current time",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            )
        )
    ],
    messages=[
        UserMessage(content="What's the weather like today in Paris? Also tell me the time in San Francisco?"),
        ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[{"name": "get_current_weather", "arguments": {"location": "Paris, France", "format": "celsius"}}, {"name": "get_current_time", "arguments": {"location": "San Francisco, CA"}}]


In [ ]:
completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_temperature",
                description="Get the current temperature in celsius",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            )
        ),
        Tool(
            function=Function(
                name="convert_temperature",
                description="Convert temperature",
                parameters={
                    "type": "object",
                    "properties": {
                        "temperature": {
                            "type": "number", # @get_current_temperature.result
                            "description": "The temperature to convert",
                        },
                        "from_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                        "to_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["temperature", "from_unit", "to_unit"],
                },
            )
        )
    ],
    messages=[
        UserMessage(content="What's the weather like today in Paris? Also convert it to fahrenheit?"),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)


[{"name": "get_current_temperature", "arguments": {"location": "Paris, France"}}, {"name": "convert_temperature", "arguments": {"temperature": 37, "from_unit": "celsius", "to_unit": "fahren


In [ ]:
completion_request = ChatCompletionRequest(
    tools=[
        Tool(
            function=Function(
                name="get_current_temperature",
                description="Get the current temperature in celsius",
                parameters={
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        }
                    },
                    "required": ["location"],
                },
            )
        ),
        Tool(
            function=Function(
                name="convert_temperature",
                description="Convert temperature",
                parameters={
                    "type": "object",
                    "properties": {
                        "temperature": {
                            "type": "number",
                            "description": "The temperature to convert",
                        },
                        "from_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                        "to_unit": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["temperature", "from_unit", "to_unit"],
                },
            )
        )
    ],
    messages=[
        SystemMessage(content="You are a helpful assistant. Your job is to select tools relevant to the user query. In the case of multiple tools, if the tools are dependent on each other, and one tool's input parameters come from another function, use @ followed by the function name for the parameter value."),
        UserMessage(content="What's the weather like today in Paris? Also tell me the time in San Francisco?"),
    ],
)

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)


[{"name": "get_current_temperature", "arguments": {"location": "Paris"}}, {"name": "convert_temperature", "arguments": {"temperature": "@get_current_temperature.result", "from_unit": "celsius", "to_unit
